In [1]:
import gymnasium
import highway_env
# import finite-mdp
# from finite_mdp.envs import finite_mdp_env

%load_ext tensorboard
import sys
from tqdm.notebook import trange
# !pip install tensorboardx gym pyvirtualdisplay
# doesn't work cause not linux
# !apt-get install -y xvfb ffmpeg
# !git clone https://github.com/Farama-Foundation/HighwayEnv.git 2> /dev/null
# !git clone https://github.com/eleurent/finite-mdp.git 2> /dev/null
sys.path.insert(0, '/content/HighwayEnv/scripts/')
# from utils import record_videos, show_videos



In [1]:
import gymnasium as gym
import random
import highway_env
from collections import defaultdict

import json
import os


# render_mode="rgb_array_list"
env = gym.make("highway-fast-v0",  config={
#    "observation": {
#     "type": "Kinematics",
#     "vehicles_count": 5,
#     "features": ["presence", "x", "y", "vx", "vy"],
#     "features_range": {
#         "x": [-100, 100],
#         "y": [-100, 100],
#         "vx": [-20, 20],
#         "vy": [-20, 20]
#     },
#     "grid_size": [[-27.5, 27.5], [-27.5, 27.5]],
#     "grid_step": [5, 5],
#     "absolute": False,
# },
 "observation": {
    "type": "TimeToCollision",
    "horizon": 5,
    # "lanes_count":2,
    # "lanes_count":2,
},
#   "action":{
#     "type": "DiscreteAction",
# }
}
)

# obs, info = env.reset()


class FiniteMarkovDecision():
    def __init__(self,env):
        self.finite_mdp = self.is_finite_mdp(env)
        self.states = env.observation_space
        if self.finite_mdp:
            self.mpd = env.mdp
        elif not self.finite_mdp:
            try:
                self.mdp = env.unwrapped.to_finite_mdp()
            except AttributeError:
                raise TypeError("not finite mdp")
            
        self.env = env
        self.obs, self.info = env.reset() 
        self.state_action_value = env.action_space
        self.load_dictionary()



    def train(self):
        gamma = 0.9  # Discount factor
        episodes = 100

        # print(self.state_action_value.n)
        for _ in range(episodes):  # Loop through each episode
            done = False
            total_reward = 0
            episode_count = 0

            # Initialize the environment and get the initial state
            self.obs = self.env.reset()  # Assuming `reset` initializes the environment
            state = str(self.obs)

            while not done:
                # Select a random action
                action = random.randrange(0, self.state_action_value.n)

                # Take the action in the environment
                next_obs, reward, done, truncated, info = self.env.step(action)

                # Accumulate discounted rewards
                total_reward += reward * (gamma ** episode_count)
                episode_count += 1

                # Update policy if the current action is better
            if state not in self.policy or self.policy[state]['reward'] < total_reward:
                self.policy[state] = {'action': action, 'reward': total_reward}

                # Update state for the next iteration
                # self.obs = next_obs
                # state = str(self.obs)
            # env.render()
            

        self.save_policy()


            
    def load_dictionary(self):
        file_path = "policies.json"
        if os.path.exists(file_path):
            try:
                with open(file_path, 'r') as file:  # Open in text mode for JSON
                    self.policy = json.load(file)
                    print("Dictionary loaded successfully.")
                    return True
            except (json.JSONDecodeError, IOError) as e:
                print(f"Error loading dictionary: {e}")
                self.policy = {}  # Use a regular dictionary as fallback
                return False
        else:
            self.policy = {}  # Use a regular dictionary if file doesn't exist
            print("No existing policy found. Starting with an empty dictionary.")
            return False

    def save_policy(self):
        file_path = "policies.json"
        try:
            # Ensure self.policy is serializable
            if isinstance(self.policy, defaultdict):
                self.policy = dict(self.policy)  # Convert defaultdict to dict

            with open(file_path, 'w') as file:  # Open in write mode
                json.dump(self.policy, file, indent=4)  # Save with pretty printing
                print("Dictionary saved successfully.")
                return True
        except IOError as e:
            print(f"Error saving dictionary: {e}")
            return False
                
    # def evaluate(self, env, episode_num):
    #     # add camera here
    #     for _ in episode_num:
    #         state = env.reset()[0]  
    #         done = False      
    #         truncated = False 
    #         # Agent navigates map until it falls into a hole (terminated), reaches goal (terminated), or has taken 200 actions (truncated).
    #         while(not done and not truncated):  
    #             # Select best action
    #             try:   
    #                 action = self.policy[str(state)]
    #             except KeyError:
    #                 action = random.randrange(0, env.action_space.n)
    #             next_state, reward, done, truncated, info = env.step(action)
    #             state = next_state
    #             # env.render()



    def is_finite_mdp(self,env):
        try:
            finite_mdp = __import__("finite_mdp.envs.finite_mdp_env")
            if isinstance(env.unwrapped, finite_mdp.envs.finite_mdp_env.FiniteMDPEnv):
                return True
        except (ModuleNotFoundError, TypeError):
            return False



finite_mdp = FiniteMarkovDecision(env)
finite_mdp.train()
# finite_mdp.evaluate(env, 1)


Dictionary loaded successfully.
Dictionary saved successfully.
